Обучите модель предсказывать значения y(i) по значениям x(i) с использованием методов RNN, LSTM, GRU.

Для этого:

1. создайте последовательности чисел, каждая из которых состоит только из цифр от 0 до 9;
2. Последовательности должны подчиняться правилу:
x - последовательность цифр, тогда:
* y1 = x1, y(i) = x(i) + x(1)
* если y(i) >= 10, то y(i) = y(i) - 10

In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

In [ ]:
# Генерация данных
def generate_data(n_samples, seq_l):
    X = np.random.randint(0, 10, size=(n_samples, seq_l))
    Y = np.zeros_like(X)

    for i in range(n_samples):
        x = X[i]
        Y[i, 0] = x[0]
        for j in range(1, seq_l):
            Y[i, j] = (x[j] + x[0]) - 10

    return X, Y

In [ ]:
# Параметры
n_samples = 10000
seq_l = 10
n_classes = 10
batch_size = 64

In [ ]:
# Подготовить данные
X, Y = generate_data(n_samples, seq_l)

X_onehot = np.eye(n_classes)[X]
Y_onehot = np.eye(n_classes)[Y]

X_tensor = torch.tensor(X_onehot, dtype=torch.float32)
Y_tensor = torch.tensor(Y_onehot, dtype=torch.float32)

split = int(0.8 * n_samples)
X_train, X_test = X_tensor[:split], X_tensor[split:]
Y_train, Y_test = Y_tensor[:split], Y_tensor[split:]

train_dataset = TensorDataset(X_train, Y_train)
test_dataset = TensorDataset(X_test, Y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Создание моделей и обучение

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out)
        return out

input_size = n_classes
hidden_size = 64
output_size = n_classes

rnn_model = RNNModel(input_size, hidden_size, output_size)

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out

lstm_model = LSTMModel(input_size, hidden_size, output_size)

In [ ]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out)
        return out

gru_model = GRUModel(input_size, hidden_size, output_size)

In [ ]:
def train(model, train_loader, test_loader, n_epochs=10, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        correct = 0
        total = 0

        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs.view(-1, n_classes), targets.view(-1, n_classes).argmax(dim=-1))
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(dim=-1)
            total += targets.size(0) * targets.size(1)
            correct += (predicted == targets.argmax(dim=-1)).sum().item()

        train_acc = correct / total

        # валидация
        model.eval()
        test_correct = 0
        test_total = 0
        with torch.no_grad():
            for inputs, targets in test_loader:
                outputs = model(inputs)
                _, predicted = outputs.max(dim=-1)
                test_total += targets.size(0) * targets.size(1)
                test_correct += (predicted == targets.argmax(dim=-1)).sum().item()

        test_acc = test_correct / test_total

        print(f'Epoch {epoch+1}/{n_epochs}, Loss: {train_loss:.3f}, Train Acc: {train_acc:.3f}, Test Acc: {test_acc:.3f}')

In [ ]:
print('RNN Model:\n')
train(rnn_model, train_loader, test_loader, n_epochs=50)

RNN Model:

Epoch 1/50, Loss: 286.124, Train Acc: 0.166, Test Acc: 0.188
Epoch 2/50, Loss: 283.815, Train Acc: 0.190, Test Acc: 0.188
Epoch 3/50, Loss: 283.249, Train Acc: 0.190, Test Acc: 0.188
Epoch 4/50, Loss: 281.236, Train Acc: 0.190, Test Acc: 0.195
Epoch 5/50, Loss: 273.041, Train Acc: 0.201, Test Acc: 0.207
Epoch 6/50, Loss: 264.711, Train Acc: 0.211, Test Acc: 0.238
Epoch 7/50, Loss: 253.729, Train Acc: 0.224, Test Acc: 0.244
Epoch 8/50, Loss: 246.231, Train Acc: 0.234, Test Acc: 0.230
Epoch 9/50, Loss: 241.470, Train Acc: 0.248, Test Acc: 0.260
Epoch 10/50, Loss: 238.575, Train Acc: 0.249, Test Acc: 0.249
Epoch 11/50, Loss: 236.883, Train Acc: 0.255, Test Acc: 0.262
Epoch 12/50, Loss: 235.657, Train Acc: 0.259, Test Acc: 0.257
Epoch 13/50, Loss: 234.710, Train Acc: 0.256, Test Acc: 0.263
Epoch 14/50, Loss: 233.955, Train Acc: 0.257, Test Acc: 0.261
Epoch 15/50, Loss: 233.201, Train Acc: 0.260, Test Acc: 0.262
Epoch 16/50, Loss: 232.437, Train Acc: 0.259, Test Acc: 0.259
Epoch

In [ ]:
print('LSTM Model:\n')
train(lstm_model, train_loader, test_loader, n_epochs=10)

LSTM Model:

Epoch 1/10, Loss: 287.191, Train Acc: 0.136, Test Acc: 0.188
Epoch 2/10, Loss: 284.890, Train Acc: 0.189, Test Acc: 0.188
Epoch 3/10, Loss: 263.990, Train Acc: 0.243, Test Acc: 0.270
Epoch 4/10, Loss: 235.962, Train Acc: 0.284, Test Acc: 0.296
Epoch 5/10, Loss: 211.862, Train Acc: 0.354, Test Acc: 0.459
Epoch 6/10, Loss: 159.976, Train Acc: 0.581, Test Acc: 0.733
Epoch 7/10, Loss: 105.366, Train Acc: 0.858, Test Acc: 0.951
Epoch 8/10, Loss: 57.963, Train Acc: 0.986, Test Acc: 0.998
Epoch 9/10, Loss: 28.359, Train Acc: 0.999, Test Acc: 1.000
Epoch 10/10, Loss: 14.517, Train Acc: 1.000, Test Acc: 1.000


In [ ]:
print('GRU Model:\n')
train(gru_model, train_loader, test_loader, n_epochs=10)

GRU Model:

Epoch 1/10, Loss: 286.766, Train Acc: 0.162, Test Acc: 0.188
Epoch 2/10, Loss: 284.024, Train Acc: 0.190, Test Acc: 0.189
Epoch 3/10, Loss: 271.017, Train Acc: 0.226, Test Acc: 0.264
Epoch 4/10, Loss: 245.320, Train Acc: 0.269, Test Acc: 0.272
Epoch 5/10, Loss: 223.960, Train Acc: 0.301, Test Acc: 0.329
Epoch 6/10, Loss: 198.829, Train Acc: 0.380, Test Acc: 0.425
Epoch 7/10, Loss: 175.384, Train Acc: 0.486, Test Acc: 0.549
Epoch 8/10, Loss: 141.994, Train Acc: 0.631, Test Acc: 0.739
Epoch 9/10, Loss: 100.517, Train Acc: 0.837, Test Acc: 0.931
Epoch 10/10, Loss: 60.440, Train Acc: 0.971, Test Acc: 0.990


Проверка моделей

In [ ]:
def predict_sequence(model, sequence):
    model.eval()
    with torch.no_grad():
        sequence_onehot = torch.tensor(np.eye(n_classes)[sequence], dtype=torch.float32).unsqueeze(0)
        outputs = model(sequence_onehot)
        _, predicted = outputs.max(dim=-1)
        return predicted.squeeze().numpy()

In [ ]:
sequence = [3, 7, 2, 8]
predicted = predict_sequence(rnn_model, sequence)
print('RNN Model:\n')
print(f"Последовательность: {sequence}\nПредсказание: {predicted}")

RNN Model:

Последовательность: [3, 7, 2, 8]
Предсказание: [3 0 5 1]


In [ ]:
sequence = [3, 7, 2, 8]
predicted = predict_sequence(lstm_model, sequence)
print('LSTM Model:\n')
print(f"Последовательность: {sequence}\nПредсказание: {predicted}")

LSTM Model:

Последовательность: [3, 7, 2, 8]
Предсказание: [3 0 5 1]


In [ ]:
sequence = [3, 7, 2, 8]
predicted = predict_sequence(gru_model, sequence)
print('GRU Model:\n')
print(f"Последовательность: {sequence}\nПредсказание: {predicted}")

GRU Model:

Последовательность: [3, 7, 2, 8]
Предсказание: [3 0 5 1]
